<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Course3/Week4/Text_Generation_with_a_character_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Tensorflow and other Libraries**

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import time

**Download the Shakespeare dataset**

In [2]:
path_to_file = keras.utils.get_file("shakespeare.txt", 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


**Read the data**

In [3]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")

print("There are {} characters in this dataset".format(len(text)))

There are 1115394 characters in this dataset


In [4]:
# taking a look at the first 250 characters in the text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# finding the unique characters in it
vocab = sorted(set(text))
print("There are {} unique characters".format(len(vocab)))

There are 65 unique characters


**Vectorize the Text**

In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print("{")
for char, _ in zip(char2idx, range(20)):
    print("{:4s} : {:3d}".format(repr(char), char2idx[char]))
print("}")

{
'\n' :   0
' '  :   1
'!'  :   2
'$'  :   3
'&'  :   4
"'"  :   5
','  :   6
'-'  :   7
'.'  :   8
'3'  :   9
':'  :  10
';'  :  11
'?'  :  12
'A'  :  13
'B'  :  14
'C'  :  15
'D'  :  16
'E'  :  17
'F'  :  18
'G'  :  19
}


In [8]:
# taking a look at the first 13 characters of the text mapped to integers
print(" {} characters mapped to integers > {}".format(repr(text[:13]), text_as_int[:13]))

 'First Citizen' characters mapped to integers > [18 47 56 57 58  1 15 47 58 47 64 43 52]


**Creating training examples and targets**

In [9]:
seq_length = 100

examples_per_epochs = len(text) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for c in char_dataset.take(5):
    print(idx2char[c])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr("".join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in dataset.take(1):
    print("Input data : {}".format(repr("".join(idx2char[input_example.numpy()]))))
    print("Output data: {}".format(repr("".join(idx2char[target_example.numpy()]))))

Input data : 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Output data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
for i, (input, target) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step{:4d}".format(i+1))
    print("Input : {}({})".format(input, repr(idx2char[input.numpy()])))
    print("Expected output: {}({})".format(target, repr(idx2char[target.numpy()])))

Step   1
Input : 18('F')
Expected output: 47('i')
Step   2
Input : 47('i')
Expected output: 56('r')
Step   3
Input : 56('r')
Expected output: 57('s')
Step   4
Input : 57('s')
Expected output: 58('t')
Step   5
Input : 58('t')
Expected output: 1(' ')


**Creating training batches**

In [14]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

**Build the Model**

In [15]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [37]:
def build_model(vocab_size, batch_size, rnn_units, embedding_dim):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape=[batch_size, None]),
        keras.layers.GRU(rnn_units, return_sequences=True,
                         stateful=True,
                         recurrent_initializer="glorot_uniform"),
        keras.layers.Dense(vocab_size)
    ])

    return model

In [43]:
model = build_model(vocab_size=vocab_size,
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

**Try the Model**

In [44]:
for input_example, target_example in dataset.take(1):
    predicted_example = model(input_example)

# should be batch_size, seq_length, vocab_size
print(predicted_example.shape)

(64, 100, 65)


In [45]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_4 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [46]:
sampled_indices = tf.random.categorical(predicted_example[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [47]:
sampled_indices

array([42, 58, 57,  7, 39,  1, 54, 39,  0, 41, 22, 44, 44, 39, 48, 28, 28,
       24, 46, 14,  6, 12,  9, 35, 62, 56, 63, 29, 24,  9, 27, 28,  9, 43,
       64, 18,  2, 33, 10, 52, 44, 45,  4, 13, 19,  0, 15, 50,  6, 42, 61,
       31, 11, 16,  4, 50,  1, 50, 13, 22, 55, 58,  2,  1, 23,  1, 15, 46,
       33, 59, 12, 35, 28, 47, 37, 21, 30, 20, 49,  5, 63, 63, 44,  7, 37,
       26, 22, 32, 27, 60, 47, 34, 18, 52, 29, 10, 42, 11, 29, 41])

In [48]:
print("Input text was : \n {}".format(repr("".join(idx2char[input_example[0].numpy()]))))
print("")
print("Predicted output was : \n{}".format(repr("".join(idx2char[sampled_indices]))))

Input text was : 
 '\nPOMPEY:\nVery well: you being then, if you be remembered,\ncracking the stones of the foresaid prunes'

Predicted output was : 
"dts-a pa\ncJffajPPLhB,?3WxryQL3OP3ezF!U:nfg&AG\nCl,dwS;D&l lAJqt! K ChUu?WPiYIRHk'yyf-YNJTOviVFnQ:d;Qc"


**Attaching an Optimizer and Loss function**

In [49]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_loss = loss(target_example, predicted_example)
print("Scalar loss :", example_loss.numpy().mean())

Scalar loss : 4.1751647


In [50]:
model.compile(optimizer="adam", loss=loss)

**Configure checkpoints**

In [51]:
checkpoint_dir = "/checkpoints"

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_prefix,
    save_weights_only=True)

**Executing the Training**

In [52]:
epochs = 30

model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback], verbose=1)

Epoch 1/30
172/172 [==============================] - 9s 55ms/step - loss: 2.6470
Epoch 2/30
172/172 [==============================] - 10s 56ms/step - loss: 1.9620
Epoch 3/30
172/172 [==============================] - 10s 56ms/step - loss: 1.6910
Epoch 4/30
172/172 [==============================] - 10s 56ms/step - loss: 1.5422
Epoch 5/30
172/172 [==============================] - 9s 55ms/step - loss: 1.4538
Epoch 6/30
172/172 [==============================] - 9s 55ms/step - loss: 1.3955
Epoch 7/30
172/172 [==============================] - 9s 55ms/step - loss: 1.3492
Epoch 8/30
172/172 [==============================] - 9s 55ms/step - loss: 1.3117
Epoch 9/30
172/172 [==============================] - 9s 55ms/step - loss: 1.2760
Epoch 10/30
172/172 [==============================] - 9s 55ms/step - loss: 1.2440
Epoch 11/30
172/172 [==============================] - 9s 55ms/step - loss: 1.2119
Epoch 12/30
172/172 [==============================] - 10s 55ms/step - loss: 1.1801
Epoch 13/

**Generating the Text**

In [53]:
tf.train.latest_checkpoint(checkpoint_dir)

'/checkpoints/ckpt_30'

In [54]:
model = build_model(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [55]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_5 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_5 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [56]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [57]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: No matter, here's the poor worm,
For one to tear faith, 'tis a good lord Gedest,
And with the lloke revenges awhile
Diving Edward's fault when women are so becoming, indeed!
This bastard by the hollow of it up on high.

HENRY PERCY:
Because your lordship morning excuse what they
ere I want me: though his father died,
Whose ensue of his substance of my head
And threat the glory of my part shall be awhile:
Digst put on my knighthood and therein values this?

POLIXENES:
I will, I am
to demand and girl in that dear people!
Come you to angry bitterly of your brother
Does for both my madren traitors and piecely since,
My brother Montague shall you to church.
For shame, my lord, here comes!

LUCENTIO:
Brother, how nged in the world.

NORTHUMBERLAND:
Rare I not proverbroom thee,
That any buy to hear me, sweet sir; what says Romeo, will ke now
Thy play she is a worthy fear; ye thou wilt stay with her!

SLY:
Are you a store of crowns.

PAULINA:
From what you please.

DUKE VINCENTIO:
By he